# 모델 구조

In [268]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Global State

In [269]:
from langgraph.graph import MessagesState
import pandas as pd
from dataclasses import field


class State(MessagesState):
    
    # 입력된 데이터
    input_id: str = ''  
    input_category: str = ''
    
    # 메타데이터 검색을 통해 확인한 제목과 설명
    title: str = ''   
    description: str = ''
    keyword: str = ''
    
    # LLM이 생성한 검색어 리스트
    query : list[str] = []
    
    # 검색어를 통해 검색한 전체 데이터
    search_df: pd.DataFrame = field(default_factory=pd.DataFrame)

    # 연관성 검색을 통해 확인한 topK 데이터
    relevance_df: pd.DataFrame = field(default_factory=pd.DataFrame)
    
    # 결과
    result_df: pd.DataFrame = field(default_factory=pd.DataFrame)


## 논문 - API 호출용 토큰 생성

In [270]:
import json, base64, requests
from Crypto.Cipher import AES
from datetime import datetime

class AESTestClass:
    def __init__(self, plain_txt, key):
        # iv, block_size 값은 고정
        self.iv = 'jvHJ1EFA0IXBrxxz'
        self.block_size = 16
        self.plain_txt = plain_txt
        self.key = key

    def pad(self):
        # PKCS#7 패딩
        number_of_bytes_to_pad = self.block_size - len(self.plain_txt) % self.block_size
        ascii_str = chr(number_of_bytes_to_pad)
        padding_str = number_of_bytes_to_pad * ascii_str
        return self.plain_txt + padding_str

    def encrypt(self):
        cipher = AES.new(self.key.encode('utf-8'), AES.MODE_CBC, self.iv.encode('utf-8'))
        padded_txt = self.pad()
        encrypted_bytes = cipher.encrypt(padded_txt.encode('utf-8'))
        # URL-safe Base64
        encrypted_str = base64.urlsafe_b64encode(encrypted_bytes).decode('utf-8')
        return encrypted_str
    
def call_access_token(MAC_ADDRESS, API_KEY, CLIENT_ID):
    # 맥주소
    mac = (MAC_ADDRESS or "").strip().strip('"').strip("'").upper().replace(":", "-")
    if not mac:
        raise SystemExit("MAC_ADDRESS가 비어있음")

    # datetime 생성
    dt = datetime.now().strftime('%Y%m%d%H%M%S')

    # JSON 페이로드 생성
    payload = {
        "mac_address": mac,
        "datetime": dt
    }
    plain_json = json.dumps(payload, separators=(',', ':'))

    # AES 암호화 → Base64
    aes = AESTestClass(plain_txt=plain_json, key=API_KEY)
    b64_cipher = aes.encrypt()

    # 인코딩 + 토큰 요청
    endpoint = "https://apigateway.kisti.re.kr/tokenrequest.do"
    params = {
        "accounts": b64_cipher,
        "client_id": CLIENT_ID
    }

    response = requests.get(endpoint, params=params, timeout=10)
    response.raise_for_status()

    data = response.json()
    token = data['access_token'] 
    return token

## 논문 - 함수

In [271]:
import xml.etree.ElementTree as ET
import pandas as pd
import json

def xml_to_df(xml):
    # XML 파싱
    root = ET.fromstring(xml)

    # recordList 찾기
    record_list_element = root.find('recordList')

    # 데이터를 담을 리스트
    records = []

    if record_list_element is not None:
        # 각 record에 대해 반복
        for record_element in record_list_element.findall('record'):
            record_data = {}
            # 각 item에 대해 반복
            for item_element in record_element.findall('item'):
                meta_code = item_element.get('metaCode')
                # CDATA 섹션의 텍스트 추출
                value = item_element.text.strip() if item_element.text else ''
                record_data[meta_code] = value
            records.append(record_data)

    df = pd.DataFrame(records)
    return df

def transform_query(input_query):

    query = {
        "BI": input_query,  # 전체
        # "TI": None,  # 논문명
        # "AU": None,  # 저자
        # "AB": None,  # 초록
        # "KW": None,  # 키워드
        # "PB": None,  # 출판사(발행기관)
        # "SN": None,  # ISSN
        # "BN": None,  # ISBN
        # "PY": None,  # 발행년도
        # "CN": None,  # 문헌번호
        # "DI": None   # DOI
    }

    json_query = json.dumps(query, separators=(',', ':')) 

    return json_query

## 논문 - 제목, 초록 가져오기

In [272]:
import os, requests, xmltodict

def ARTI_browse(state: State):
    
    CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
    ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
    MAC_ADDRESS = os.getenv("MAC_ADDRESS")

    access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)

    url = "https://apigateway.kisti.re.kr/openapicall.do"
    params = {
        "client_id": CLIENT_ID,
        "token": access_token,
        "version": 1.0,
        "action": "browse",
        "target": "ARTI",
        "cn": state['input_id'],
        "include": "",
        "exclude": None,
    }
    
    res = requests.get(url, params=params, timeout=20)
    xml = res.text
    dict_data = xmltodict.parse(xml)
    with open("../data/input_data.json", "w", encoding="utf-8") as f:
        json.dump(dict_data, f, ensure_ascii=False, indent=4)

    df = xml_to_df(xml)
    
    title, description, keyword = df['Title'].iloc[0], df['Abstract'].iloc[0], df['Keyword'].iloc[0]
    
    print('\n[title]\n', title)
    print('\n[description]\n', description)
    print('\n[keyword]\n', keyword)    

    return {'title': title, 'description': description, 'keyword': keyword}


## 데이터셋 - 제목, 설명 가져오기

In [273]:
import os, requests, json

def DATA_browse(state: State):

    API_KEY = os.getenv("DATAON_META_API_KEY")
    assert API_KEY and API_KEY.strip(), "환경변수(DATAON_META_API_KEY)가 비어있어요!"

    url = "https://dataon.kisti.re.kr/rest/api/search/dataset/" + state["input_id"]
    params = {"key": API_KEY}

    res = requests.get(url, params=params, timeout=20)
    data = res.json()

    # json 저장
    with open("../data/input_data.json", "w", encoding="utf-8") as f:
        json.dump(data['records'], f, ensure_ascii=False, indent=4)

    title, description, keyword = data['records']['dataset_title_etc_main'], data['records']['dataset_expl_etc_main'], data['records']['dataset_kywd_etc_main']
    print('\n[title]\n', title)
    print('\n[description]\n', description)
    print('\n[keyword]\n', keyword)

    return {'title':title, 'description' : description, 'keyword': keyword} 

## 검색어 생성

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from typing_extensions import Annotated
from pydantic import Field, BaseModel

# Schema
class QueryResult(BaseModel):
    query: Annotated[
        list[str],
        Field(
            ..., 
            max_length=5, 
            min_length=3,
            description="가장 적절한 검색어들의 리스트, 길이 최소 3개/최대 5개", 
        )
    ]
    
# Prompt
query_template = '''
주어진 제목과 설명을 바탕으로, 의미적으로 가장 관련성이 높은 논문과 데이터셋을 찾기 위한 검색어를 생성하세요.

[검색 엔진 제약]
검색은 정확 일치 기반입니다. 쿼리는 짧고 응집력 있게 만드세요.
각 쿼리는 2~3단어, 고유명사나 기술 토픽의 조합을 선호합니다.

[생성 절차]
1. 주제 핵심어 파악: 연구 주제의 주요 객체, 방법론, 도메인, 응용 맥락을 한 문장으로 요약
2. 후보 생성: 2~3단어 쿼리 후보를 8~10개 잠정 생성
3. 필터링 규칙 적용:
  - 일반어, 지나치게 포괄적이거나 모호한 표현 제거(ex: “AI model”, “data analysis”)
  - 12개는 방법론 중심, 12개는 도메인 중심, 1~2개는 응용 시나리오 중심으로 균형 있게 남김
  - 약어만 있는 경우는 배제하되, 널리 쓰이는 고유 약어는 유지(ex: “BERT” 가능, “ML” 단독 불가)
  - 하이픈, 특수문자, 따옴표는 사용하지 않음
4. 최종 선택: 상위 3~5개만 남김

[금지]
- 1단어 쿼리 금지
- 4단어 이상 금지
- 불용어만 남는 조합 금지(ex: “for research”)
- JSON 스키마 위반 금지

[Input]
- 연구 주제: {title}
- 연구 설명: {description}
- 키워드: {keyword}

[Output]
다음 형식의 JSON을 출력하세요:
{{
  "query": [],
}}
'''

query_prompt = PromptTemplate.from_template(query_template)

# Node
def generate_query(state: State):

    prompt = query_prompt.invoke(
        {
            'title': state['title'], 
            'description': state['description'],
            'keyword': state['keyword'],
        }
    )

    # sLLM
    sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    structured_sllm = sllm.with_structured_output(QueryResult)
    res = structured_sllm.invoke(prompt)
    query = res.query
    print('\n[query]\n', query)

    return {'query': query}


## 논문 - 쿼리에 따른 검색

In [275]:
import os, requests

def ARTI_search(state: State):
    
    CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
    ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
    MAC_ADDRESS = os.getenv("MAC_ADDRESS")

    access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)

    url = "https://apigateway.kisti.re.kr/openapicall.do"
    df = pd.DataFrame()
    
    for query in state['query']:
        params = {
            "client_id": CLIENT_ID,
            "token": access_token,
            "version": 1.0,
            "action": "search",
            "target": "ARTI",
            "searchQuery": transform_query(query),
            'curPage': 1, # 현재페이지 번호
            'rowCount': 20, # 디스플레이 건수(기본값 10, 최대값 100)
        }

        res = requests.get(url, params=params, timeout=20)
        xml = res.text
        tmp = xml_to_df(xml)
        tmp["query"] = query
        df = pd.concat([df, tmp], ignore_index=True)
        
    df = df.drop_duplicates(subset='CN')
    df.to_csv('../data/search_results_article.csv', index=False, encoding='utf-8')
    print('\n[total article length]\n', len(df))

    cleaned_df = (
        df[
            ['CN', 'Title', 'Abstract', 'Pubyear', 'Keyword', 'Author', 'ContentURL', 'query']
        ]
        .rename(
            columns={
                'CN': 'ID',
                'Title': 'title',
                'Abstract': 'description',
                'Pubyear': 'pubyear',
                'Keyword': 'keyword',
                'Author': 'author',
                'ContentURL': 'URL'
            }
        )
    )

    cleaned_df['category'] = 'article'

    search_df = state.get('search_df')
    
    if search_df is not None and not search_df.empty:
        return {'search_df': pd.concat([search_df, cleaned_df], ignore_index=True)}
    else:
        return {'search_df': cleaned_df}


## 데이터셋 - 쿼리에 따른 검색

In [276]:
import os, requests
import pandas as pd

def DATA_search(state: State):
    
    API_KEY = os.getenv("DATAON_SEARCH_API_KEY")
    assert API_KEY and API_KEY.strip(), "환경변수(DATAON_API_KEY)가 비어있어요!"

    url = "https://dataon.kisti.re.kr/rest/api/search/dataset/"
    df = pd.DataFrame()
    for query in state['query']:
        params = {"key": API_KEY, "query": query, "from": 0, "size": 20}
        # key / CHAR / 필수 / API_KEY
        # query / CHAR / 필수 / 검색키워드
        # from / CHAR / 옵션 / 페이지시작위치
        # size / CHAR / 옵션 / 페이지사이즈

        res = requests.get(url, params=params, timeout=20)
        data = res.json()
        
        if "records" in data:
            tmp = pd.DataFrame(data["records"])
            tmp["query"] = query
            df = pd.concat([df, tmp], ignore_index=True)

    df = df.drop_duplicates(subset='svc_id')
    df.to_csv('../data/search_results_dataset.csv', index=False, encoding='utf-8')
    print('\n[total dataset length]\n', len(df))
    
    cleaned_df = (
        df[
            ['svc_id', 'dataset_title_etc_main', 'dataset_expl_etc_main','dataset_pub_dt_pc', 'dataset_kywd_etc_main', 'dataset_creator_etc_main', 'dataset_lndgpg', 'query']
        ]
        .rename(
            columns={
                'svc_id': 'ID',
                'dataset_title_etc_main': 'title',
                'dataset_expl_etc_main': 'description',
                'dataset_pub_dt_pc': 'pubyear',
                'dataset_kywd_etc_main': 'keyword',
                'dataset_creator_etc_main': 'author',
                'dataset_lndgpg': 'URL',
            }
        )
    )

    cleaned_df['category'] = 'dataset'
    
    search_df = state.get('search_df')
    
    if search_df is not None and not search_df.empty:
        return {'search_df': pd.concat([search_df, cleaned_df], ignore_index=True)}
    else:
        return {'search_df': cleaned_df}


## 연관성 점수 부여

In [277]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from tqdm import tqdm
import pandas as pd

def evaluate_relevance(state: State):
    
    df = state['search_df']

    targets = ['title', 'description', 'keyword']

    MAX_LENGTH = 2000

    dfs = {}

    for target in targets:
        print(f'\n[embedding_{target}]')
        
        df[target] = df[target].fillna('')
        df[target] = df[target].str.slice(0, MAX_LENGTH)

        texts = df[target].tolist()

        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

        docs = [Document(page_content=text, metadata={"ID": row.ID}) 
                for text, row in zip(texts, df.itertuples())]

        batch_size = 500
        stores = []
        for i in tqdm(range(0, len(docs), batch_size)):
            batch = docs[i:i+batch_size]
            store = FAISS.from_documents(batch, embeddings)
            stores.append(store)

        vectorstore = stores[0]
        for s in stores[1:]:
            vectorstore.merge_from(s)

        query = state[target]
        query_embedding = embeddings.embed_query(query)

        results_with_score = vectorstore.similarity_search_with_score_by_vector(query_embedding, k=20)
        
        dfs[f'df_{target}'] = pd.DataFrame([
            {
                "ID": r.metadata.get("ID"),
                "relevance": score,
                "target": target
            }
            for r, score in results_with_score
        ])
        
    merged_df = dfs['df_title'].merge(
        dfs['df_description'][["ID", "relevance"]], 
        on="ID",
        how="outer",
        suffixes=("_title", "_desc")
    ).merge(
        dfs['df_keyword'][["ID", "relevance"]].rename(columns={"relevance": "relevance_key"}), 
        on="ID",
        how="outer"
    )

    merged_df = merged_df.fillna(2.0)

    merged_df = merged_df.drop_duplicates(subset='ID')

    merged_df = merged_df[merged_df['ID'] != state['input_id']]

    # 2. 가중치 합산
    a, b, c = 10, 3, 1
    merged_df["relevance_raw"] = (
        merged_df["relevance_title"] * a + 
        merged_df["relevance_desc"] * b + 
        merged_df["relevance_key"] * c
    ) / (a + b + c)

    merged_df["relevance"] = 100 * (1 - merged_df["relevance_raw"] / 2)

    result_df = (merged_df[["ID", "relevance"]]
                .sort_values("relevance", ascending=False)
                .reset_index(drop=True)).head(5)

    return {'relevance_df': result_df}

## 추천사유 생성

In [278]:
from typing_extensions import Annotated
from pydantic import Field, BaseModel
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Schema
class IDRelevance(BaseModel):
    relevant_id: Annotated[
        list[str],
        Field(
            ..., 
            description=(
                "데이터의 ID 목록"
            ), 
        )
    ]
    reason: Annotated[
        list[str],
        Field(
            ..., 
            description="각 ID가 선정된 이유를 설명하는 문자열 목록. relevant_id와 인덱스가 일치해야 합니다.",
        )
    ]

# Prompt
reason_template = '''
당신은 데이터 과학자입니다. 아래는 연구 데이터 목록입니다.

각 데이터 혹은 논문 항목은 다음 컬럼을 가지고 있습니다:
- ID: 각 데이터의 고유키
- 제목
- 설명
- 키워드

[목표]
모든 항목에 대해 해당 항목의 선정 사유를 작성해 주세요.

**중요: relevant_id와 reason의 개수는 반드시 동일해야 합니다.**

[Input]
연구 주제: {title}
연구 설명: {description}
키워드: {keyword}

[Data]
데이터 목록:
{data}

[Output]
다음 형식의 JSON을 출력하세요. relevant_id와 reason의 길이가 정확히 일치해야 합니다:

{{
  "relevant_id": ["ID1", "ID2", "ID3"],
  "reason": ["이유1", "이유2", "이유3"]
}}
'''

reason_prompt = PromptTemplate.from_template(reason_template)

# Node
def generate_reason(state: State):
    print('\ngenerating reasons...\n')
    df, title, description, keyword = state['search_df'], state['title'], state['description'], state['keyword']
    
    relevant_ids = state['relevance_df']['ID'].tolist()
    filtered_df = df[df['ID'].isin(relevant_ids)]

    prompt = reason_prompt.invoke(
        {
            'title': title, 
            'description': description,
            'keyword': keyword,
            'data': filtered_df[['ID', 'title', 'description', 'keyword']].to_dict(orient="records"),
        }
    )

    sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    structured_sllm = sllm.with_structured_output(IDRelevance)
    res = structured_sllm.invoke(prompt)
        
    tmp = pd.DataFrame({
        'ID': res.relevant_id,
        'reason': res.reason
    })
    
    relevance_df = pd.merge(
        state['relevance_df'],
        tmp,
        on='ID',
        how='left'
    )
    
    return {'relevance_df': relevance_df}

## 결과 정리

In [279]:
def summarize_results(state: State):
    
    merged_df = pd.merge(
        state['relevance_df'],
        state['search_df'][['ID','category', 'title', 'description', 'URL']],
        on='ID',
        how='left'
    )

    result_df = merged_df[['category', 'title', 'description', 'relevance', 'reason', 'URL']]
    result_df = result_df.rename(columns={
        'category': '구분',
        'title': '제목',
        'description': '설명',
        'relevance': '점수',
        'reason': '추천 사유',
    })

    return {'result_df': result_df}

# 그래프 구조

In [280]:
def input_router(state):

    if state['input_category'] == 'article':
        return 'article'
    
    if state['input_category'] == 'dataset':
        return 'dataset'

    return 'END'

In [281]:
from langgraph.graph import START, END, StateGraph

def build_graph():
    builder = StateGraph(State)
    
    builder.add_sequence([generate_query, ARTI_search, DATA_search, evaluate_relevance, generate_reason, summarize_results]) 
    builder.add_node('DATA_browse', DATA_browse)
    builder.add_node('ARTI_browse', ARTI_browse)
    
    builder.add_conditional_edges(
        START,
        input_router,
        {
            'article': 'ARTI_browse',
            'dataset': 'DATA_browse'
        }
    )
    
    builder.add_edge('ARTI_browse', 'generate_query')
    builder.add_edge('DATA_browse', 'generate_query')
    
    builder.add_edge('summarize_results', END)
    
    return builder.compile()

graph = build_graph()

In [19]:
# input_id = 'JAKO200411922932805'
# input_category = 'article'

input_id = '37c0f3d51a130211fe55fe6019cc7914'
input_category = 'dataset'

print('[input_id]\n', input_id)

res = graph.invoke({
    'input_id': input_id,
    'input_category': input_category,
})

display(res['result_df'])

[input_id]
 37c0f3d51a130211fe55fe6019cc7914

[title]
 Gravity Core from Antarctic ROSS Sea (RS15-GC76)

[description]
 2014/2015 Gravity core, Ross Sea (O.Granite Harbor), Antarctic Climate change observation. Location : 76°54.9315'S, 163°21.7774'E Water Depth : 808m Core Length : 3.17m

[keyword]
 EARTH SCIENCE;OCEANS;MARINE SEDIMENTS;SEDIMENTATION

[query]
 ['Antarctic Gravity Core', 'Ross Sea Sediments', 'Marine Sedimentation', 'Climate Change Antarctica', 'Earth Science Core']

[total article length]
 100

[total dataset length]
 46

[embedding_title]


100%|██████████| 1/1 [00:00<00:00,  1.07it/s]



[embedding_description]


100%|██████████| 1/1 [00:01<00:00,  1.12s/it]



[embedding_keyword]


100%|██████████| 1/1 [00:01<00:00,  1.56s/it]



generating reasons...



,구분,제목,설명,점수,추천 사유,URL
0,dataset,Gravity Core from Antarctic ROSS Sea (RS15-GC77),"2014/2015 Gravity core, Ross Sea (O.Granite Ha...",99.775948,"이 데이터는 O.Granite Harbor 지역에서 수집된 중력 코어로, 연구 주제...",https://dx.doi.org/doi:10.22663/KOPRI-KPDC-000...
1,dataset,Gravity Core from Antarctic ROSS Sea (RS15-GC80),"2014/2015 Gravity core, Ross Sea (O.Granite Ha...",99.454041,"이 데이터는 Ross Sea에서의 중력 코어 샘플로, 연구 주제와 동일한 지역에서 ...",https://dx.doi.org/doi:10.22663/KOPRI-KPDC-000...
2,dataset,Gravity Core from Antarctic ROSS Sea (RS15-GC79),"2014/2015 Gravity core, Ross Sea, Antarctic Cl...",99.090996,"이 데이터는 Ross Sea에서의 중력 코어 샘플로, 연구 주제와 유사한 기후 변화...",https://dx.doi.org/doi:10.22663/KOPRI-KPDC-000...
3,dataset,Gravity Core from Antarctic ROSS Sea (RS15-GC72),"2014/2015 Gravity core, Ross Sea (NE.Lewis Bay...",98.839508,"이 데이터는 Ross Sea의 NE.Lewis Bay 지역에서 수집된 중력 코어로,...",https://dx.doi.org/doi:10.22663/KOPRI-KPDC-000...
4,dataset,Gravity Core from Antarctic ROSS Sea (RS15-GC78),"2014/2015 Gravity core, Ross Sea (Mawson Glaci...",98.637108,이 데이터는 Ross Sea의 Mawson Glacier 지역에서 수집된 중력 코어...,https://dx.doi.org/doi:10.22663/KOPRI-KPDC-000...
